<a href="https://colab.research.google.com/github/YannPhamVan/StockMarketsAnalyticsZoomcamp/blob/main/02-dataframe-analysis-homework/02-dataframe-analysis-homework-Q1-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Module 2 Homework (2025 Cohort)

In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK](https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb) covered at the livestream to re-use the code snippets.

---
#0) Imports and Installs

In [1]:
!pip install yfinance

In [2]:
# IMPORTS
import numpy as np
import pandas as pd
import requests


#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

---
### Question 1: [IPO] Withdrawn IPOs by Company Type

**What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?**

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn](https://stockanalysis.com/ipos/withdrawn/)), collect and process the data to find out which company type saw the most withdrawn IPO value.

#### Steps:
1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   *It is a similar process to Code Snippet 1 discussed at the livestream.*    You should get **99 entries**.

In [3]:
import pandas as pd
import requests
from io import StringIO

def get_withdrawn_ipos() -> pd.DataFrame:
    """
    Fetch withdrawn IPO data from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [4]:
withdrawn_ipos = get_withdrawn_ipos()
withdrawn_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [5]:
withdrawn_ipos

,Symbol,Company Name,Price Range,Shares Offered
0,ODTX,"Odyssey Therapeutics, Inc.",-,-
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000
2,AURN,"Aurion Biotech, Inc.",-,-
3,ROTR,"PHI Group, Inc.",-,-
4,ONE,One Power Company,-,-
...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-
96,CHO,Chobani Inc.,-,-
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231
98,GLGX,"Gerson Lehrman Group, Inc.",-,-


2. Create a new column called `Company Class`, categorizing company names based on patterns like:
   - “Acquisition Corp” or “Acquisition Corporation” → `Acq.Corp`
   - “Inc” or “Incorporated” → `Inc`
   - “Group” → `Group`
   - “Ltd” or “Limited” → `Limited`
   - “Holdings” → `Holdings`
   - Others → `Other`

  *  Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited', you assign the 'Limited' class).

  * Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.


In [6]:
def get_company_class(company_name: str):
  ''' Apply a company name to get its class.
  '''
  cn = company_name.lower()
  if cn.find('acquisition corp')>=0 or cn.find('acquisition corporation')>=0:
    return 'Acq.Corp'
  elif cn.find('inc')>=0 or cn.find('incorporated')>=0:
    return 'Inc'
  elif cn.find('group')>=0:
    return 'Group'
  elif cn.find('ltd')>=0 or cn.find('limited')>=0:
    return 'Limited'
  elif cn.find('holdings')>=0:
    return 'Holdings'
  else:
    return 'Other'

withdrawn_ipos["Company Class"] = withdrawn_ipos["Company Name"].apply(get_company_class)

withdrawn_ipos

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc


In [7]:
withdrawn_ipos.value_counts('Company Class')

,count
Company Class,
Inc,51
Acq.Corp,21
Limited,17
Other,6
Group,4
Holdings,1


In [8]:
withdrawn_ipos.loc[withdrawn_ipos['Company Class'] == 'Other']

,Symbol,Company Name,Price Range,Shares Offered,Company Class
4,ONE,One Power Company,-,-,Other
9,KMCM,Key Mining Corp.,$2.25,4444444,Other
53,CLLB,"CoLabs Intâl, Corp.",$4.50,1300000,Other
74,TSIV,Twelve Seas Investment Company IV TMT,$10.00,20000000,Other
86,FSPR,Four Springs Capital Trust,$13.00 - $15.00,18000000,Other
99,HCG,hear.com N.V.,$17.00 - $20.00,16220000,Other



3. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price Range` column). Examples:
   - '$8.00-$10.00' → `9.0`  
   - '$5.00' → `5.0`  
   - '-' → `None`

In [9]:
def get_avg_price(price_range: str):
  '''
  Compute average price from price range.
  '''
  pr = price_range.replace('$', '')
  if pr == '-':
    return None
  elif pr.find('-') >= 0:
    return (float(pr.split('-')[0]) + float(pr.split('-')[1])) / 2
  else:
    return float(pr)

withdrawn_ipos["Avg. Price"] = withdrawn_ipos["Price Range"].apply(get_avg_price)

withdrawn_ipos

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. Price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN



4. Convert `Shares Offered` to numeric, clean missing or invalid values.

In [10]:
withdrawn_ipos["Shares Offered"] = pd.to_numeric(withdrawn_ipos["Shares Offered"], errors='coerce')
withdrawn_ipos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Symbol          100 non-null    object 
 1   Company Name    100 non-null    object 
 2   Price Range     100 non-null    object 
 3   Shares Offered  72 non-null     float64
 4   Company Class   100 non-null    object 
 5   Avg. Price      73 non-null     float64
dtypes: float64(2), object(4)
memory usage: 4.8+ KB



5. Create a new column:  
   `Withdrawn Value = Shares Offered * Avg Price` (**71 non-null values**)

In [11]:
withdrawn_ipos["Withdrawn Value"] = withdrawn_ipos["Shares Offered"] * withdrawn_ipos["Avg. Price"]
withdrawn_ipos.notna().sum()

,0
Symbol,100
Company Name,100
Price Range,100
Shares Offered,72
Company Class,100
Avg. Price,73
Withdrawn Value,71



6. Group by `Company Class` and calculate total withdrawn value.

In [12]:
withdrawn_ipos.groupby('Company Class').agg({'Withdrawn Value': 'sum'})

,Withdrawn Value
Company Class,
Acq.Corp,4.021000e+09
Group,3.378750e+07
Holdings,7.500000e+07
Inc,2.257164e+09
Limited,5.497346e+08
Other,7.679200e+08



7. **Answer**: Which class had the highest **total** value of withdrawals?
---

In [13]:
withdrawn_ipos.groupby('Company Class').agg({'Withdrawn Value': 'sum'}).sort_values('Withdrawn Value', ascending=False).iloc[0]

,Acq.Corp
Withdrawn Value,4.021000e+09


---
### Question 2:   [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)


**What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?**

The goal is to replicate the large-scale `yfinance` OHLCV data download and perform basic financial calculations on IPO stocks.


#### Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from:  
   [https://stockanalysis.com/ipos/2024/](https://stockanalysis.com/ipos/2024/)  
   Filter to keep only those IPOs **before 1 June 2024** (first 5 months of 2024).  
   ➤ You should have **75 tickers**.


In [14]:
def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [15]:
ipos_2024 = get_ipos_by_year(2024)
ipos_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      225 non-null    object
 1   Symbol        225 non-null    object
 2   Company Name  225 non-null    object
 3   IPO Price     225 non-null    object
 4   Current       225 non-null    object
 5   Return        225 non-null    object
dtypes: object(6)
memory usage: 10.7+ KB


In [16]:
ipos_2024["IPO Date"] = pd.to_datetime(ipos_2024["IPO Date"], format='mixed')

In [33]:
ipos_2024["IPO Price"] = ipos_2024["IPO Price"].apply(get_avg_price)
ipos_2024

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2024-12-31,ONEG,OneConstruction Group Limited,4.00,$4.90,20.55%
1,2024-12-27,PHH,"Park Ha Biological Technology Co., Ltd.",4.00,$25.58,498.75%
2,2024-12-23,HIT,"Health In Tech, Inc.",4.00,$0.61,-84.75%
3,2024-12-23,TDAC,Translational Development Acquisition Corp.,10.00,$10.26,2.60%
4,2024-12-20,RANG,Range Capital Acquisition Corp.,10.00,$10.21,2.00%
...,...,...,...,...,...,...
220,2024-01-18,CCTG,CCSC Technology International Holdings Limited,6.00,$1.06,-82.17%
221,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,16.45,$14.63,-11.85%
222,2024-01-12,SYNX,Silynxcom Ltd.,4.00,$1.89,-56.63%
223,2024-01-11,SDHC,Smith Douglas Homes Corp.,21.00,$18.50,-12.10%


In [35]:
ipos_first_5_months_2024 = ipos_2024.loc[ipos_2024['IPO Date'] < '2024-06-01']
ipos_first_5_months_2024.dropna(inplace=True)
ipos_first_5_months_2024

/tmp/ipython-input-35-1440939620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipos_first_5_months_2024.dropna(inplace=True)


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
149,2024-05-23,BOW,Bowhead Specialty Holdings Inc.,17.00,$36.36,112.18%
150,2024-05-17,HDL,Super Hi International Holding Ltd.,19.56,$18.70,-4.76%
151,2024-05-17,RFAI,RF Acquisition Corp II,10.00,$10.53,5.30%
152,2024-05-15,JDZG,JIADE Limited,4.00,$0.29,-92.77%
153,2024-05-15,RAY,Raytech Holding Limited,4.00,$1.33,-70.00%
...,...,...,...,...,...,...
220,2024-01-18,CCTG,CCSC Technology International Holdings Limited,6.00,$1.06,-82.17%
221,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,16.45,$14.63,-11.85%
222,2024-01-12,SYNX,Silynxcom Ltd.,4.00,$1.89,-56.63%
223,2024-01-11,SDHC,Smith Douglas Homes Corp.,21.00,$18.50,-12.10%


2.  Use **Code Snippet 7** to download daily stock data for those tickers (via `yfinance`).  
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.  
   Define a new column `growth_252d` representing growth after **252 trading days** (~1 year), in addition to any other growth periods you already track.

In [36]:
ALL_TICKERS = ipos_first_5_months_2024["Symbol"].to_list()
ALL_TICKERS

['BOW',
 'HDL',
 'RFAI',
 'JDZG',
 'RAY',
 'BTOC',
 'ZK',
 'GPAT',
 'PAL',
 'SVCO',
 'NNE',
 'CCIX',
 'VIK',
 'ZONE',
 'LOAR',
 'MRX',
 'RBRK',
 'NCI',
 'MFI',
 'YYGH',
 'TRSG',
 'CDTG',
 'CTRI',
 'IBTA',
 'MTEN',
 'TWG',
 'ULS',
 'PACS',
 'MNDR',
 'CTNM',
 'MAMO',
 'ZBAO',
 'BOLD',
 'MMA',
 'UBXG',
 'IBAC',
 'AUNA',
 'BKHA',
 'LOBO',
 'RDDT',
 'ALAB',
 'INTJ',
 'RYDE',
 'LGCL',
 'SMXT',
 'VHAI',
 'DYCQ',
 'CHRO',
 'UMAC',
 'HLXB',
 'MGX',
 'TBBB',
 'TELO',
 'KYTX',
 'PMNT',
 'AHR',
 'LEGT',
 'ANRO',
 'GUTS',
 'AS',
 'FBLG',
 'AVBP',
 'BTSG',
 'HAO',
 'CGON',
 'YIBO',
 'JL',
 'SUGP',
 'JVSA',
 'KSPI',
 'CCTG',
 'PSBD',
 'SYNX',
 'SDHC',
 'ROMA']

In [37]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90, 252, 365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [38]:
stocks_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_90d,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,1.092017,NaN,NaN,0,0.089496,NaN,1
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,0.998054,NaN,NaN,0,0.084436,NaN,0
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,1.001133,NaN,NaN,0,0.068165,NaN,1
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,0.987828,NaN,NaN,0,0.037619,NaN,0
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,NaN,NaN,NaN,1.037490,NaN,NaN,0,0.065417,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23311,2.870000,2.890000,2.560000,2.660000,123100,0.0,0.0,ROMA,2025,6,...,3.917526,4.666667,NaN,NaN,3.1970,2.82155,1,0.124060,10.383358,0
23312,2.840000,3.000000,2.640000,2.875000,63100,0.0,0.0,ROMA,2025,6,...,4.342900,5.424529,NaN,NaN,3.1485,2.86180,1,0.125217,9.939668,0
23313,2.850000,2.935000,2.790000,2.795000,10200,0.0,0.0,ROMA,2025,6,...,4.092240,5.008960,NaN,NaN,3.0870,2.89955,1,0.051878,9.469241,0
23314,2.883000,2.900000,2.730000,2.790000,33600,0.0,0.0,ROMA,2025,6,...,4.182909,5.157116,NaN,NaN,2.9900,2.93555,1,0.060932,9.196931,0





3. Calculate the Sharpe ratio assuming a risk-free rate of **4.5%**:

   ```python
   stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']
   ```

   ⚠️ **IMPORTANT** Please use the original version of annualized volatility calculation (it was later corrected to another formula):
   ```python
   stocks_df['volatility'] =   stocks_df['Close'].rolling(30).std() * np.sqrt(252)
   ```

In [39]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']


4. Filter the DataFrame to keep data only for the trading day:  
   **‘2025-06-06’**

   Compute descriptive statistics (e.g., `.describe()`) for these columns:  
   - `growth_252d`  
   - `Sharpe`

   You should observe:  
   - `growth_252d` is defined for **71 out of 75 stocks** (some IPOs are too recent or data starts later).  
   - Median `growth_252d` is approximately **0.75** (indicating a 25% decline), while mean is about **1.15**, showing a bias towards high-growth companies pushing the average up.

In [40]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], format='mixed')

In [41]:
stocks_df.loc[stocks_df['Date'] == '2025-06-06'].describe()[["growth_252d", "Sharpe"]]

,growth_252d,Sharpe
count,71.000000,71.000000
mean,1.152897,0.288285
min,0.024970,-0.079677
25%,0.293422,0.041215
50%,0.758065,0.083768
75%,1.362736,0.311507
max,8.097413,2.835668
std,1.406017,0.519028




5. **Answer:**  
   - What is the **median Sharpe ratio** for these 71 stocks?  **0.083768**
   - Note: Positive `Sharpe` means growth exceeding the risk-free rate of 4.5%.  
   - [Additional] Do you observe the **same top 10 companies** when sorting by `growth_252d` versus sorting by `Sharpe`? **Not really as only one Ticker is common to the 2 lists**

---

In [42]:
stocks_df.loc[stocks_df['Date'] == '2025-06-06'].sort_values(by="growth_252d", ascending=False).head(10)["Ticker"]

,Ticker
20480,JL
23306,ROMA
14286,UMAC
2957,NNE
4647,RBRK
16676,AHR
18023,AS
4358,MRX
11515,RDDT
6987,MTEN


In [43]:
stocks_df.loc[stocks_df['Date'] == '2025-06-06'].sort_values(by="Sharpe", ascending=False).head(10)["Ticker"]

,Ticker
10891,BKHA
21154,JVSA
16984,LEGT
10303,IBAC
14627,HLXB
8179,MNDR
13613,DYCQ
12143,INTJ
20480,JL
5815,TRSG


---
### Question 3: [IPO] ‘Fixed Months Holding Strategy’

**What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?**  
(*Assume you buy at the close of the first trading day and sell after a fixed number of trading days.*)


---

#### Goal:
Investigate whether holding an IPO stock for a fixed number of months after its first trading day produces better returns, using future growth columns.

---

#### Steps:

1. **Start from the existing DataFrame** from Question 2 (75 tickers from IPOs in the first 5 months of 2024).  

   Add **12 new columns**:  
   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`  
   *(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)*  
   This logic is similar to `historyPrices['growth_future_30d']` from **Code Snippet 7**, but extended to longer timeframes.

In [44]:
def compute_future_growth(df, months=12):
    df_sorted = df.sort_values(['Ticker', 'Date'])
    df_sorted = df_sorted.set_index('Date')

    for m in range(1, months+1):
        df_sorted[f'growth_future_{m}m'] = (
            df_sorted.groupby('Ticker')['Close'].shift(-21 * m) / df_sorted['Close']
        )
    return df_sorted.reset_index()

future_growth_df = compute_future_growth(stocks_df)
future_growth_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
0,2024-02-07,12.085784,12.471401,11.878868,12.433780,12732800,0.0,0.0,AHR,2024,...,1.060660,1.125316,1.214961,1.308118,1.753552,1.963886,1.988395,2.257204,2.259752,2.273312
1,2024-02-08,12.245675,12.452590,12.226864,12.283296,1630300,0.0,0.0,AHR,2024,...,1.094691,1.156243,1.267090,1.324937,1.817034,1.951132,2.105590,2.256847,2.197812,2.230109
2,2024-02-09,12.236269,12.650101,12.179837,12.339726,2062100,0.0,0.0,AHR,2024,...,1.086583,1.157160,1.262084,1.352007,1.803991,1.927870,2.077638,2.219440,2.180527,2.207051
3,2024-02-12,13.120364,13.694087,12.320916,12.367942,1161200,0.0,0.0,AHR,2024,...,1.090294,1.151425,1.278880,1.390634,1.864410,1.953675,2.103896,2.231068,2.194799,2.278196
4,2024-02-13,12.367943,12.753560,12.236270,12.556049,1631700,0.0,0.0,AHR,2024,...,1.067100,1.131888,1.289178,1.399258,1.905472,1.933801,2.028534,2.218782,2.205361,2.222738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23311,2025-06-13,3.710000,3.710000,3.170000,3.290000,79800,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23312,2025-06-16,3.120000,3.290000,2.839000,3.080000,97000,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23313,2025-06-17,2.990000,3.240000,2.915000,3.160000,32700,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23314,2025-06-18,3.160000,3.460000,3.078000,3.230000,77300,0.0,0.0,ZONE,2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. **Determine the first trading day** (`min_date`) for each ticker.  
   This is the earliest date in the data for each stock.

In [45]:
min_date_df = future_growth_df.groupby('Ticker').Date.agg('min').reset_index()
min_date_df

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
...,...,...
70,YIBO,2024-01-25
71,YYGH,2024-04-22
72,ZBAO,2024-04-02
73,ZK,2024-05-10






3. **Join the data**:  
   Perform an **inner join** between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.  
   ➤ You should end up with **75 records** (one per IPO) with all 12 `future_growth_...` fields populated.

In [46]:
joined_df = future_growth_df.merge(min_date_df, on=['Ticker', 'Date'], how='inner')
joined_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
0,2024-02-07,12.085784,12.471401,11.878868,12.433780,12732800,0.0,0.0,AHR,2024,...,1.060660,1.125316,1.214961,1.308118,1.753552,1.963886,1.988395,2.257204,2.259752,2.273312
1,2024-03-20,52.560001,63.500000,50.610001,62.029999,16843300,0.0,0.0,ALAB,2024,...,1.018217,0.853780,0.631307,0.759794,1.078027,1.457037,1.974367,1.982105,1.381912,1.189424
2,2024-02-02,22.000000,23.270000,20.000000,20.700001,2386300,0.0,0.0,ANRO,2024,...,0.686957,0.546377,0.563768,0.438164,0.647343,0.492270,0.192271,0.203865,0.222705,0.202899
3,2024-02-01,13.400000,13.800000,13.100000,13.400000,18656400,0.0,0.0,AS,2024,...,1.080597,1.143284,0.935075,0.802239,0.999254,1.236567,1.332090,1.950000,2.152985,2.362687
4,2024-03-22,9.510000,10.320000,9.300000,9.600000,9046900,0.0,0.0,AUNA,2024,...,0.801042,0.919792,0.782292,0.767708,0.723958,0.733333,0.706250,0.857292,0.850000,0.752083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2024-01-25,4.000000,4.000000,2.400000,2.790000,875500,0.0,0.0,YIBO,2024,...,0.734767,0.788530,0.723656,1.044803,0.777778,0.957706,0.996416,1.290323,1.326165,1.842294
71,2024-04-22,3.880000,4.700000,2.580000,2.900000,990300,0.0,0.0,YYGH,2024,...,0.246897,0.255172,0.315517,0.431034,0.631034,0.675862,0.634483,0.606897,0.617241,0.355172
72,2024-04-02,4.000000,4.000000,3.000000,3.700000,755000,0.0,0.0,ZBAO,2024,...,1.075676,1.037838,0.891892,1.005405,0.851351,0.770270,0.424324,0.418919,0.370270,0.300541
73,2024-05-10,26.000000,29.360001,25.510000,28.260000,9709800,0.0,0.0,ZK,2024,...,0.508139,0.598726,0.869073,0.878627,0.998231,0.961076,0.877212,1.105096,0.756546,1.018401




4. **Compute descriptive statistics** for the resulting DataFrame:  
   Use `.describe()` or similar to analyze each of the 12 columns:  
   - `future_growth_1m`  
   - `future_growth_2m`  
   - ...  
   - `future_growth_12m`  

In [47]:
joined_df.describe()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Year,Month,...,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
count,75,75.000000,75.000000,75.000000,75.000000,7.500000e+01,75.0,75.0,75.0,75.0000,...,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,74.000000,74.000000,71.000000
mean,2024-03-24 07:40:48,15.618374,17.843774,14.310103,15.856867,5.341061e+06,0.0,0.0,2024.0,3.2800,...,0.833824,0.825086,0.803768,0.864185,0.847149,0.832981,0.881776,0.917942,0.882532,0.900859
min,2024-01-09 00:00:00,0.010700,0.015700,0.006200,0.011600,6.000000e+02,0.0,0.0,2024.0,1.0000,...,0.060947,0.045368,0.054109,0.061432,0.044086,0.043103,0.033144,0.037769,0.023674,0.038947
25%,2024-02-08 12:00:00,4.770000,5.700000,3.602500,4.140000,7.332000e+05,0.0,0.0,2024.0,2.0000,...,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.242424,0.264661,0.229211
50%,2024-03-28 00:00:00,10.000000,10.320000,10.000000,10.040000,1.992600e+06,0.0,0.0,2024.0,3.0000,...,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.772592,0.717585,0.634667
75%,2024-04-25 00:00:00,21.875000,23.430000,19.125000,19.900000,5.451900e+06,0.0,0.0,2024.0,4.0000,...,1.069085,1.134300,1.016381,1.093948,1.114468,1.082346,1.049719,1.200647,1.106726,1.136392
max,2024-09-11 00:00:00,103.250000,103.500000,95.029999,98.000000,4.870550e+07,0.0,0.0,2024.0,9.0000,...,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711
std,NaN,17.547395,19.277085,16.693328,18.236340,8.882742e+06,0.0,0.0,0.0,1.6649,...,0.409763,0.401969,0.488348,0.653179,0.712943,0.762422,0.936956,0.911430,0.863182,0.892906




5. **Determine the best holding period**:  
   - Find the number of months **(1 to 12)** where the **average (mean)** future growth is **maximal**.  
   - This optimal month shows an uplift of **>1%** compared to all others.  
   - Still, the average return remains **less than 1** (i.e., expected return is less than doubling your investment).

**Answer = 2**

In [48]:
joined_df[joined_df.columns[-12:]].describe().iloc[1].sort_values(ascending=False)

,mean
growth_future_2m,0.940544
growth_future_1m,0.927259
growth_future_10m,0.917942
growth_future_12m,0.900859
growth_future_11m,0.882532
growth_future_9m,0.881776
growth_future_6m,0.864185
growth_future_7m,0.847149
growth_future_3m,0.833824
growth_future_8m,0.832981
